# ДЗ №3, Волжина Елена
[Матрицы весов](https://compscicenter.ru/learning/assignments/24203/)


![](hw03.jpg)

In [1]:
from collections import defaultdict

match = {
    'A': {'A':  5, 'R': -2, 'N': -1, 'K': -1},
    'R': {'A': -2, 'R':  7, 'N': -1, 'K':  3},
    'N': {'A': -1, 'R': -1, 'N':  7, 'K':  0},
    'K': {'A': -1, 'R':  3, 'N':  0, 'K':  6},
}
D = 100     # gap
string1, string2 = "AKRANR", "KAAANK"

# выравнивание без gap   -1 -1 -2 +5 +7 +3 = 11

In [2]:
def align(s1, s2):
    assert s1 and s2, "one of strings is empty!"
    
    n, m = len(s1), len(s2)
    sources = defaultdict(lambda: defaultdict(list))
    weights = [[None for _ in range(m + 1)] for _ in range(n + 1)]
    
    # first row
    for n_col in range(m + 1):
        weights[0][n_col] = 0
        sources[0][n_col] = [(0, 0)]
        
    # first column
    for n_row in range(n + 1):
        weights[n_row][0] = 0
        sources[n_row][0] = [(0, 0)]
    
    sources[0][0] = []
    
    def get_ways(n_row, n_col):
        c1, c2 = s1[n_row - 1], s2[n_col - 1]
        return [
            ((n_row - 1, n_col - 1), match[c1][c2]),  # (mis)match
            ((n_row - 1, n_col), -D),   # move down
            ((n_row, n_col - 1), -D),   # move right
        ]
    
    # dynamically fill the rest
    def process(n_row, n_col):
        for (old_n_row, old_n_col), diff in get_ways(n_row, n_col):
            prev_value = weights[old_n_row][old_n_col]
            new_value = prev_value + diff
            
            if weights[n_row][n_col] is None or weights[n_row][n_col] < new_value:
                weights[n_row][n_col] = new_value
                sources[n_row][n_col] = [(old_n_row, old_n_col)]
            elif weights[n_row][n_col] == new_value:
                sources[n_row][n_col].append((old_n_row, old_n_col))
    
    for k in range(1, min(n, m) + 1):
        process(k, k)
        for n_row in range(k + 1, n + 1):
            process(n_row, k)
        for n_col in range(k + 1, m + 1):
            process(k, n_col)
        
    return weights, sources

In [3]:
alignment_weights, alignment_sources = align(string1, string2)

In [4]:
print("WEIGHTS MATRIX")
print("\n".join(" ".join("{: >3}".format(x) for x in row) 
                for row in alignment_weights))

WEIGHTS MATRIX
  0   0   0   0   0   0   0
  0  -1   5   5   5  -1  -1
  0   6  -2   4   4   5   5
  0   3   4  -4   2   3   8
  0  -1   8   9   1   1   2
  0   0  -2   7   8   8   1
  0   3  -2  -4   5   7  11


In [7]:
def restore_paths(sources, n_row, n_col):
    if sources[n_row][n_col]:
        result = []
        for source in sources[n_row][n_col]:
            paths = restore_paths(sources, *source)
            result.extend([p + [(n_row, n_col)] for p in paths])
        return result
    else:
        return [[(n_row, n_col)]]


alignment_paths = restore_paths(alignment_sources, len(string1), len(string2))
alignment_paths

[[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6)]]

In [8]:
def print_alignments(paths):
    for path in paths:
        res1, res2 = [], []
        for (r1, c1), (r2, c2) in zip(path, path[1:]):
            if r1 + 1 == r2 and c1 + 1 == c2:
                # mis(match)
                res1.append(string1[r2 - 1])
                res2.append(string2[c2 - 1])
            elif r1 + 1 == r2 and c1 == c2:
                # down
                res1.append(string1[r2 - 1])
                res2.append("-")
            else:
                # right
                res1.append("-")
                res2.append(string2[c2 - 1])
        print(res1, "\n", res2, "\n\n", sep="")

print("GLOBAL ALIGNMENTS:")
print_alignments(alignment_paths)

GLOBAL ALIGNMENTS:
['A', 'K', 'R', 'A', 'N', 'R']
['K', 'A', 'A', 'A', 'N', 'K']


